In [1]:
!/usr/bin/python3 -m pip install --upgrade pip
!pip install keras-rl2
!pip install gym

In [3]:
!pip install stable-baseline3[extra]

ERROR: Could not find a version that satisfies the requirement stable-baseline3[extra] (from versions: none)
ERROR: No matching distribution found for stable-baseline3[extra]


In [2]:
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras

In [3]:
class TriaTempEnv(Env):
    def __init__(self):
        #actions we can take is on, off.
        self.action_space = Discrete(2)
        #------
        self.observation_space = gym.spaces.Dict(
            spaces ={
                "T": spaces.Box(low=40, high=80, shape=(1,), dtype=np.float64),
                "H": spaces.Box(low=30, high=90, shape=(1,), dtype=np.float64),
                "A": spaces.Box(low=0, high=300, shape=(1,), dtype=np.float64),
            })
        #-----
        # temperature array
        self.observation_space = Box(low=np.array([0.0]), high=np.array([100.0]))
        
        # set starting temperature
        self.state = 40.0 + random.randint(-2,2)
        
        # set tria simulation duration
        self.tria_length= 100
        
    def step(self, action):
        # apply action
        self.state +=  action - 1
        
        #reduce tria simulation length by 1 second
        self.tria_length -= 1
        
        #claculate reward
        if self.state >= 50 and self.state <= 80:
            reward = 1 
        else: 
            reward = -1
        
        if self.tria_length <= 0:
            done = True
        else:
            done = False
            
        # apply temperature noise    
        self.state += random.randint(-1,1)
        
        #set placeholder
        info = {}
        
        return self.state, reward, done, info
            
        
    def render(self):
        pass
    
    def reset(self):
        self.state = 40 + random.randint(-2,2)
        
        self.tria_length= 100
        
        return self.state
        

In [4]:
env = TriaTempEnv()

/usr/local/lib/python3.8/dist-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [5]:
episodes = 10
for episode in range (1, episodes + 1):
    state = env.reset()
    done = False
    score = 0
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
        print('Episode: {} score: {}'.format(episode, score))
    

Episode: 1 score: -1
Episode: 1 score: -2
Episode: 1 score: -3
Episode: 1 score: -4
Episode: 1 score: -5
Episode: 1 score: -6
Episode: 1 score: -7
Episode: 1 score: -8
Episode: 1 score: -9
Episode: 1 score: -10
Episode: 1 score: -11
Episode: 1 score: -12
Episode: 1 score: -13
Episode: 1 score: -14
Episode: 1 score: -15
Episode: 1 score: -16
Episode: 1 score: -17
Episode: 1 score: -18
Episode: 1 score: -19
Episode: 1 score: -20
Episode: 1 score: -21
Episode: 1 score: -22
Episode: 1 score: -23
Episode: 1 score: -24
Episode: 1 score: -25
Episode: 1 score: -26
Episode: 1 score: -27
Episode: 1 score: -28
Episode: 1 score: -29
Episode: 1 score: -30
Episode: 1 score: -31
Episode: 1 score: -32
Episode: 1 score: -33
Episode: 1 score: -34
Episode: 1 score: -35
Episode: 1 score: -36
Episode: 1 score: -37
Episode: 1 score: -38
Episode: 1 score: -39
Episode: 1 score: -40
Episode: 1 score: -41
Episode: 1 score: -42
Episode: 1 score: -43
Episode: 1 score: -44
Episode: 1 score: -45
Episode: 1 score: -

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [7]:
states = env.observation_space.shape
actions = env.action_space.n

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=states))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
del model

In [16]:
model = build_model(states, actions)

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                32        
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 3)                 51        
                                                                 
Total params: 627
Trainable params: 627
Non-trainable params: 0
_________________________________________________________________


In [18]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
          nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [20]:
from tensorflow.keras.optimizers.legacy import Adam
#opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.1)
#Adam._name = 'hey'
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

#dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)  

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 25:27 - reward: -1.0000

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.8/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 189s 19ms/step - reward: 0.0710
100 episodes - episode_reward: 7.100 [-100.000, 74.000] - loss: 0.852 - mae: 4.204 - mean_q: 8.679

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 190s 19ms/step - reward: 0.6526
100 episodes - episode_reward: 65.260 [-2.000, 90.000] - loss: 3.087 - mae: 18.541 - mean_q: 30.908

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 192s 19ms/step - reward: 0.7294
100 episodes - episode_reward: 72.940 [24.000, 90.000] - loss: 6.062 - mae: 28.963 - mean_q: 46.427

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 215s 22ms/step - reward: 0.6860
100 episodes - episode_reward: 68.600 [30.000, 88.000] - loss: 8.161 - mae: 33.159 - mean_q: 51.318

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 228s 23ms/step - reward: 0.5316
done, took 1014.844 seconds


In [21]:
scores = dqn.test(env,nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 84.000, steps: 100
Episode 2: reward: 78.000, steps: 100
Episode 3: reward: 78.000, steps: 100
Episode 4: reward: 82.000, steps: 100
Episode 5: reward: 80.000, steps: 100
Episode 6: reward: 80.000, steps: 100
Episode 7: reward: 84.000, steps: 100
Episode 8: reward: 90.000, steps: 100
Episode 9: reward: 84.000, steps: 100
Episode 10: reward: 84.000, steps: 100
Episode 11: reward: 80.000, steps: 100
Episode 12: reward: 74.000, steps: 100
Episode 13: reward: 86.000, steps: 100
Episode 14: reward: 72.000, steps: 100
Episode 15: reward: 78.000, steps: 100
Episode 16: reward: 88.000, steps: 100
Episode 17: reward: 84.000, steps: 100
Episode 18: reward: 86.000, steps: 100
Episode 19: reward: 82.000, steps: 100
Episode 20: reward: 78.000, steps: 100
Episode 21: reward: 80.000, steps: 100
Episode 22: reward: 70.000, steps: 100
Episode 23: reward: 80.000, steps: 100
Episode 24: reward: 80.000, steps: 100
Episode 25: reward: 90.000, steps: 100
Episo